In [2]:
# https://pandas.pydata.org/pandas-docs/stable/reference/frame.html
# pivot_table就是转换各个维度去观察数据, aggfunc就是在数据转换过程中的过程函数
import pandas as pd
import numpy as np

In [3]:
df = pd.DataFrame({'student': ['小红', '小红', '李华', '李华', '小天', '小天'],
                    'class': ['001','001','001','001','002','002'],
                   'subject': ['C', 'Java', 'Python', 'C', 'C', 'Python'],
                   'grades': [80,  90, 78, 90, 80, 78]})

In [4]:
# 显示pivot_table语法参数
?df.pivot_table

In [7]:
df

,student,class,subject,grades
0,小红,001,C,80
1,小红,001,Java,90
2,李华,001,Python,78
3,李华,001,C,90
4,小天,002,C,80
5,小天,002,Python,78


In [41]:
# 只显示grades的值，因为student,class都不是数字，是字符串，所以无法进行mean。
print(df.pivot_table(index='subject'))

            grades
subject           
C        83.333333
Java     90.000000
Python   78.000000


In [42]:
# index作为纵轴索引、columns作为横轴索引来观察指定的values值，另外aggfunc指定的是均值函数（mean）
print(df.pivot_table(columns='subject'))

subject          C  Java  Python
grades   83.333333  90.0    78.0


In [43]:
# index、columns参数必须传一个，否则会报以下错误：ValueError: No group keys passed!

In [44]:
# aggfunc接收的对象是DataFrame
df.pivot_table(index='subject', aggfunc=lambda x: type(x))

,class,grades,student
subject,,,
C,<class 'pandas.core.frame.DataFrame'>,<class 'pandas.core.frame.DataFrame'>,<class 'pandas.core.frame.DataFrame'>
Java,<class 'pandas.core.frame.DataFrame'>,<class 'pandas.core.frame.DataFrame'>,<class 'pandas.core.frame.DataFrame'>
Python,<class 'pandas.core.frame.DataFrame'>,<class 'pandas.core.frame.DataFrame'>,<class 'pandas.core.frame.DataFrame'>


In [45]:
# tolist() 转换成列表
# 通过指定维度后透视得到的值的列表
# 所以mean函数在作用于class、student这两列是字符串元素的列表肯定是不对的，所以被过滤掉了。
df.pivot_table(index='subject', aggfunc=lambda x: x.tolist())

,class,grades,student
subject,,,
C,"[001, 001, 002]","[80, 90, 80]","[小红, 李华, 小天]"
Java,[001],[90],[小红]
Python,"[001, 002]","[78, 78]","[李华, 小天]"


In [46]:
# （1）统计各个班级(class)的平均分
df.pivot_table(index='class')

,grades
class,
001,84.5
002,79.0


In [47]:
# reset_index()重置索引
# 关于 reset_index() https://www.cnblogs.com/keye/p/11229863.html
df.pivot_table(index='class').reset_index()

,class,grades
0,001,84.5
1,002,79.0


In [48]:
# 不想保留原来的index，使用参数 drop=True，默认 False。
# 使用后，class索引被删除
df.pivot_table(index='class').reset_index(drop=True)

,grades
0,84.5
1,79.0


In [49]:
# （2）统计各个班级(class)的平均分以及班级学生人数
# df.pivot_table(index='class', aggfunc={'grades': np.mean, 'student': lambda x: len(x.unique())})  等同于
df.pivot_table(index='class', aggfunc={'grades': np.mean, 'student': lambda x: x.nunique()})

,grades,student
class,,
001,84.5,2
002,79.0,1


In [50]:
df.student.unique()

array(['小红', '李华', '小天'], dtype=object)

In [51]:
df.student.nunique()

3

In [52]:
# （3）统计各个班级(class)的各个科目(subject)的平均分
df.pivot_table(index='class', columns='subject',  values='grades')

subject,C,Java,Python
class,,,
001,85.0,90.0,78.0
002,80.0,NaN,78.0


In [56]:
# （4）统计各个班级(class)的各个科目(subject)的最高分(空值填充为0)
df.pivot_table(index='class', columns='subject',  values='grades', aggfunc=max, fill_value=0)

subject,C,Java,Python
class,,,
001,90,90,78
002,80,0,78


In [58]:
# （5）统计各个班级(class)的各个科目(subject)的人数(空值填充为0)
df.pivot_table(index='class', columns='subject',  values='grades', aggfunc='count', fill_value=0)

subject,C,Java,Python
class,,,
001,2,1,1
002,1,0,1


In [82]:
# （6）统计各个学生(student)的最高分,最低分,平均分
df.pivot_table(index='student', values='grades', aggfunc=[max, min, np.mean])

,max,min,mean
,grades,grades,grades
student,,,
小天,80,78,79
小红,90,80,85
李华,90,78,84


In [84]:
# （7）统计各个学生(student)的每个科目的最高分,最低分,平均分
df.pivot_table(index='student', columns='subject', values='grades', aggfunc=[max, min, np.mean],fill_value=0)

max             min             mean            
subject   C Java Python   C Java Python    C Java Python
student                                                 
小天       80    0     78  80    0     78   80    0     78
小红       80   90      0  80   90      0   80   90      0
李华       90    0     78  90    0     78   90    0     78

In [62]:
# 上面是多级索引,可能你想去掉grades这一级, 可以参考下面方法
# stack()是行转列, 把grades从column变成了index, 再reset_index去掉grades
df.pivot_table(index='student', values='grades', aggfunc=[max, min, np.mean]).stack()

,,max,min,mean
student,,,,
小天,grades,80,78,79
小红,grades,90,80,85
李华,grades,90,78,84


In [77]:
# reset_index(level= -1|0|1)表示索引列显示的位置
df.pivot_table(index='student', values='grades', aggfunc=[max, min, np.mean]).stack().reset_index(level=1)

,level_1,max,min,mean
student,,,,
小天,grades,80,78,79
小红,grades,90,80,85
李华,grades,90,78,84


In [80]:
df.pivot_table(index='student', values='grades', aggfunc=[max, min, np.mean]).stack().reset_index(level=1,drop=True)

,max,min,mean
student,,,
小天,80,78,79
小红,90,80,85
李华,90,78,84


In [81]:
# （8）获取各个学生(student)的最高分的level, 其中划分方式是: "C" < 80 <= "B" < 90 <= "A"
import bisect

def get_grade_level(series):
    max_grade = max(series)
    return 'CBA'[bisect.bisect_right([80, 90], max_grade)]

df.pivot_table(index='student', values='grades', aggfunc=get_grade_level)

,grades
student,
小天,B
小红,A
李华,A
